# Data cleaning and preparation for analyzing NYC taxi data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df1=pd.read_csv('2017_taxi_trips.csv')
df2=pd.read_csv('2018_taxi_trips.csv')
df3=pd.read_csv('2019_taxi_trips.csv')
df4=pd.read_csv('2020_taxi_trips.csv')

C:\Users\Rufus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df1.head(2)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type
0,1,2017-01-01 09:00:01.000,2017-01-01 09:03:56.000,N,1,74,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
1,1,2017-01-01 18:57:55.000,2017-01-01 19:01:16.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0


In [6]:
df4.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [7]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11740667, 18)
(8807303, 18)
(6044050, 19)
(1734051, 19)


1. Let’s stick to trips that were NOT sent via “store and forward”

In [8]:
df1=df1[df1['store_and_fwd_flag']=='N']
df2=df2[df2['store_and_fwd_flag']=='N']
df3=df3[df3['store_and_fwd_flag']=='N']
df4=df4[df4['store_and_fwd_flag']=='N']

In [9]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11723594, 18)
(8790612, 18)
(5615484, 19)
(1201260, 19)


2. I’m only interested in street-hailed trips paid by card or cash

In [10]:
df1=df1[df1['payment_type'].isin([1,2])]
df2=df2[df2['payment_type'].isin([1,2])]
df3=df3[df3['payment_type'].isin([1,2])]
df4=df4[df4['payment_type'].isin([1,2])]

In [11]:
df2['payment_type'].unique()

array([2, 1], dtype=int64)

3. We can remove any trips with dates before 2017 or after 2020, along with any trips with pickups or drop-offs into unknown zones

In [12]:
df4['PULocationID'].nunique()

257

4. Let’s assume any trips with no recorded passengers had 1 passenger

In [13]:
df1['passenger_count'].replace(0,1,inplace=True)
df2['passenger_count'].replace(0,1,inplace=True)
df3['passenger_count'].replace(0,1,inplace=True)
df4['passenger_count'].replace(0,1,inplace=True)

In [14]:
# checking if there are any daatframes with passenger count still as 0
df4['passenger_count'].unique()

array([1., 2., 3., 5., 6., 7., 4., 8., 9.])

5. We can remove any trips with distance and fare amount as zero

In [15]:
# function to drop rows where trip distance and fare amounts are 0
def dropdistfare(dff):
    dff2=dff[(dff['trip_distance']==0) & (dff['fare_amount']==0)]
    l=dff2.index
    dff.drop(l,axis=0,inplace=True)
    return dff


In [16]:
df1=dropdistfare(df1)
df2=dropdistfare(df2)
df3=dropdistfare(df3)
df4=dropdistfare(df4)

6. If you notice any records where the fare, taxes, and surcharges are ALL negative, please make them positive

In [17]:
df1[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]=df1[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']].abs()
df2[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]=df2[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']].abs()
df3[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]=df3[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']].abs()
df4[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]=df4[['fare_amount', 'extra', 'mta_tax','tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']].abs()

7. For any trips that have a fare amount but have a trip distance of 0, calculate the distance this way: (Fare amount - 2.5) / 2.5

In [18]:
def functdistfare2(dff):
    dff2=dff[(dff['fare_amount']>2.5) & (dff['trip_distance']==0)]
    l=dff2.index
    for i in l:
        dff.loc[i,'trip_distance']=(dff.loc[i,'fare_amount']-2.5)/2.5
    return dff

In [19]:
### Below is a much efficient and faster method 

In [20]:
dff_1=df1[(df1['fare_amount']!=0) & (df1['trip_distance']==0)]
l=dff_1.index    
df1.loc[l,'trip_distance']=(df1.loc[l,'fare_amount']-2.5)/2.5

In [21]:
dff_2=df2[(df2['fare_amount']!=0) & (df2['trip_distance']==0)]
l=dff_2.index    
df2.loc[l,'trip_distance']=(df2.loc[l,'fare_amount']-2.5)/2.5

In [22]:
df_4=df4[(df4['fare_amount']!=0) & (df4['trip_distance']==0)]
l=df_4.index    
df4.loc[l,'trip_distance']=(df4.loc[l,'fare_amount']-2.5)/2.5

In [23]:
df_3=df3[(df3['fare_amount']!=0) & (df3['trip_distance']==0)]
l=df_3.index    
df3.loc[l,'trip_distance']=(df3.loc[l,'fare_amount']-2.5)/2.5

In [24]:
df1[(df1['fare_amount']!=0) & (df1['trip_distance']==0)]['fare_amount'].unique()

# we get value as 2.5 as (2.5-2.5)/2.5 is zero 

array([2.5])

8. For any trips that have a trip distance but have a fare amount of 0, calculate the fare amount this way: 2.5 + (trip distance x 2.5)

In [25]:
# let's check a random df first 
df1[(df1['trip_distance']!=0) & (df1['fare_amount']==0)]['VendorID'].count()

18624

In [26]:
df1_copy=df1[(df1['trip_distance']!=0) & (df1['fare_amount']==0)]
l=df1_copy.index
df1.loc[l,'fare_amount']=(df1.loc[l,'trip_distance']+2.5)*2.5

In [27]:
df1[(df1['trip_distance']!=0) & (df1['fare_amount']==0)]['VendorID'].count()

0

In [28]:
df2_copy=df2[(df2['trip_distance']!=0) & (df2['fare_amount']==0)]
l=df2_copy.index
df2.loc[l,'fare_amount']=(df2.loc[l,'trip_distance']+2.5)*2.5

In [29]:
df2[(df2['trip_distance']!=0) & (df2['fare_amount']==0)]['VendorID'].count()

0

In [30]:
df3_copy=df3[(df3['trip_distance']!=0) & (df3['fare_amount']==0)]
l=df3_copy.index
df3.loc[l,'fare_amount']=(df3.loc[l,'trip_distance']+2.5)*2.5

In [31]:
df3[(df3['trip_distance']!=0) & (df3['fare_amount']==0)]['VendorID'].count()

0

In [32]:
df4_copy=df4[(df4['trip_distance']!=0) & (df4['fare_amount']==0)]
l=df4_copy.index
df4.loc[l,'fare_amount']=(df4.loc[l,'trip_distance']+2.5)*2.5

In [33]:
df4[(df4['trip_distance']!=0) & (df4['fare_amount']==0)]['VendorID'].count()

0

In [34]:
df1['trip_distance']=df1['trip_distance'].abs()

In [35]:
df2['trip_distance']=df2['trip_distance'].abs()
df3['trip_distance']=df3['trip_distance'].abs()
df4['trip_distance']=df4['trip_distance'].abs()

In [36]:
# we need to add another column 'congestion_surcharge' to df1 and df2

df1['congestion_surcharge']=0
df2['congestion_surcharge']=0

10. If a pickup date/time is AFTER the drop-off date/time, let’s swap them

In [37]:
df1_copy=df1[df1['lpep_pickup_datetime'] > df1['lpep_dropoff_datetime']]
l=df1_copy.index
x=df1.loc[l,'lpep_dropoff_datetime']
df1.loc[l,'lpep_dropoff_datetime']=df1.loc[l,'lpep_pickup_datetime']
df1.loc[l,'lpep_pickup_datetime']=x

In [38]:
df1[df1['lpep_pickup_datetime'] > df1['lpep_dropoff_datetime']]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge


In [39]:
df2_copy=df2[df2['lpep_pickup_datetime'] > df2['lpep_dropoff_datetime']]
l=df2_copy.index
x=df2.loc[l,'lpep_dropoff_datetime']
df2.loc[l,'lpep_dropoff_datetime']=df2.loc[l,'lpep_pickup_datetime']
df2.loc[l,'lpep_pickup_datetime']=x

In [40]:
df2[df2['lpep_pickup_datetime'] > df2['lpep_dropoff_datetime']]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge


In [41]:
df3_copy=df3[df3['lpep_pickup_datetime'] > df3['lpep_dropoff_datetime']]
l=df3_copy.index
x=df3.loc[l,'lpep_dropoff_datetime']
df3.loc[l,'lpep_dropoff_datetime']=df3.loc[l,'lpep_pickup_datetime']
df3.loc[l,'lpep_pickup_datetime']=x

In [42]:
df3[df3['lpep_pickup_datetime'] > df3['lpep_dropoff_datetime']]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge


In [43]:
df4_copy=df4[df4['lpep_pickup_datetime'] > df4['lpep_dropoff_datetime']]
l=df4_copy.index
x=df4.loc[l,'lpep_dropoff_datetime']
df4.loc[l,'lpep_dropoff_datetime']=df4.loc[l,'lpep_pickup_datetime']
df4.loc[l,'lpep_pickup_datetime']=x

In [44]:
df4[df4['lpep_pickup_datetime'] > df4['lpep_dropoff_datetime']]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge


11. Adding the pickup zone

In [45]:
pickup_df=pd.read_csv('taxi_zones.csv')

In [46]:
pickup_df.rename(columns = {'location_ID':'PULocationID'}, inplace = True)

In [47]:
pickup_df.columns

Index(['PULocationID', 'pickup_zone'], dtype='object')

In [48]:
# before the joins happen 
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11633875, 19)
(8727494, 19)
(5570368, 19)
(1190588, 19)


In [49]:
df1=pd.merge(df1,pickup_df,how='inner',on='PULocationID')
df2=pd.merge(df2,pickup_df,how='inner',on='PULocationID')
df3=pd.merge(df3,pickup_df,how='inner',on='PULocationID')
df4=pd.merge(df4,pickup_df,how='inner',on='PULocationID')

In [50]:
#after joining
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11633875, 20)
(8727494, 20)
(5570368, 20)
(1190588, 20)


12. Adding the dropoff zones

In [51]:
dropoff_df=pd.read_csv('taxi_zones_drop.csv')

In [52]:
dropoff_df.rename(columns = {'location_ID':'DOLocationID'}, inplace = True)

In [53]:
dropoff_df.columns

Index(['DOLocationID', 'dropoff_zone'], dtype='object')

In [54]:
# before the joins happen 
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11633875, 20)
(8727494, 20)
(5570368, 20)
(1190588, 20)


In [55]:
df1=pd.merge(df1,dropoff_df,how='inner',on='DOLocationID')
df2=pd.merge(df2,dropoff_df,how='inner',on='DOLocationID')
df3=pd.merge(df3,dropoff_df,how='inner',on='DOLocationID')
df4=pd.merge(df4,dropoff_df,how='inner',on='DOLocationID')

In [56]:
#after joining
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(11633875, 21)
(8727494, 21)
(5570368, 21)
(1190588, 21)


### Concatinating the 4 dfs

In [57]:
new_df=pd.concat([df1,df2,df3,df4])

In [ ]:
new_df.shape

In [ ]:
new_df.head()

In [ ]:
# saving new_df as a csv file for analysis in Tableau

new_df.to_csv('taxi_data_final.csv')

In [63]:
# saving them as individual datasets too 

df1.to_csv('taxi_df1_2017.csv')
df2.to_csv('taxi_df2_2018.csv')
df3.to_csv('taxi_df3_2019.csv')
df4.to_csv('taxi_df4_2020.csv')

In [58]:
new_df.shape

(27122325, 21)

In [60]:
pd.set_option('display.max_columns', None)
new_df.head(2)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,pickup_zone,dropoff_zone
0,1.0,2017-01-01 09:00:01.000,2017-01-01 09:03:56.000,N,1.0,74,41,1.0,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2.0,1.0,0.0,East Harlem North,Central Harlem
1,2.0,2017-01-02 14:34:17.000,2017-01-02 14:39:29.000,N,1.0,74,41,1.0,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2.0,1.0,0.0,East Harlem North,Central Harlem
